# Azure AI Search semantic chunking using Markdown and Document Intelligence

This code demonstrates how to use Azure AI Search with [Document Intelligence semantic chunking with Markdown](https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-retrieval-augmented-generation?view=doc-intel-4.0.0) and the Azure AI Search Documents Python SDK.

It uses `azd` and a bicep template for all deployment steps so that you can focus on queries.

## Prerequisites

+ Follow the instructions in the [readme](./readme.md) to deploy all Azure resources, and to create and load the search index.

+ Check your search service to make sure the index exists. If you don't see an index, revisit the readme and run the `setup_search_service` script.

+ Don't add an `.env` file to this folder. Environment variables are read from the `azd` deployment.

+ Install the packages necessary for running the queries in this notebook. 

In [ ]:
! pip uninstall python-dotenv

In [1]:
! pip install azure-search-documents==11.6.0b4 --quiet
! pip install python-dotenv azure-identity --quiet

In [1]:
# Load all environment variables from the azd deployment
import subprocess
from io import StringIO
from dotenv import load_dotenv
result = subprocess.run(["azd", "env", "get-values"], stdout=subprocess.PIPE)
load_dotenv(stream=StringIO(result.stdout.decode("utf-8")))

True

In [2]:
import os
search_url = f"https://{os.environ['AZURE_SEARCH_SERVICE']}.search.windows.net"

## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [14]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.identity import DefaultAzureCredential
# Pure Vector Search
query = "What's a performance review?"  
  
search_client = SearchClient(search_url, "document-intelligence-index", credential=DefaultAzureCredential())
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
#vector_query = RawVectorQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text= query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk_headers", "chunk"],
    top=1
)  
for result in results:
    print(f"parent_id: {result['parent_id']}")  
    print(f"Score: {result['@search.score']}") 
    print(f"Chunk Headers: {result['chunk_headers']}")
    print(f"Content: {result['chunk']}")  


parent_id: aHR0cHM6Ly9zdGd5b2xhNG5kbDNqZGkuYmxvYi5jb3JlLndpbmRvd3MubmV0L2RvY3VtZW50LWludGVsbGlnZW5jZS1zYW1wbGUtZGF0YS9lbXBsb3llZV9oYW5kYm9vay5wZGY1
Score: 0.03333333507180214
Chunk Headers: ['Contoso Electronics Employee Handbook', 'Values', 'Performance Reviews', 'Performance Reviews at Contoso Electronics']
Content: ### Performance Reviews  
#### Performance Reviews at Contoso Electronics  
At Contoso Electronics, we strive to ensure our employees are getting the feedback they
need to continue growing and developing in their roles. We understand that performance
reviews are a key part of this process and it is important to us that they are conducted in an
effective and efficient manner.  
Performance reviews are conducted annually and are an important part of your career
development. During the review, your supervisor will discuss your performance over the
past year and provide feedback on areas for improvement. They will also provide you with
an opportunity to discuss your goals and

## Perform a hybrid search

Search using text and vectors combined for more relevant results

In [8]:
# Hybrid Search
query = "What's the difference between the health plans?"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk_headers", "chunk"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Chunk Headers: {result['chunk_headers']}")
    print(f"Content: {result['chunk']}")  


HttpResponseError: () Authorization failed.
Code: 
Message: Authorization failed.

## Use chunk headers to improve search results

Semantic chunking retrieves section headers if they are available. Use them to improve your search results

Note that semantic chunking from document intelligence automatically converts tables to Markdown form

In [12]:
# Hybrid Search
query = "How does the cost between health plans compare?"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk_headers", "chunk"],
    search_fields=["chunk", "chunk_headers"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Chunk Headers: {result['chunk_headers']}")
    print(f"Content: {result['chunk']}")  


parent_id: aHR0cHM6Ly9zdGd5b2xhNG5kbDNqZGkuYmxvYi5jb3JlLndpbmRvd3MubmV0L2RvY3VtZW50LWludGVsbGlnZW5jZS1zYW1wbGUtZGF0YS9CZW5lZml0X09wdGlvbnMucGRm0
chunk_id: cef761850e8b_aHR0cHM6Ly9zdGd5b2xhNG5kbDNqZGkuYmxvYi5jb3JlLndpbmRvd3MubmV0L2RvY3VtZW50LWludGVsbGlnZW5jZS1zYW1wbGUtZGF0YS9CZW5lZml0X09wdGlvbnMucGRm0_chunks_4
Score: 0.03333333507180214
Chunk Headers: ['Contoso Electronics', 'Cost Comparison']
Content: ## Cost Comparison  
Contoso Electronics deducts the employee's portion of the healthcare cost from each paycheck. This
means that the cost of the health insurance will be spread out over the course of the year, rather
than being paid in one lump sum. The employee's portion of the cost will be calculated based on the
selected health plan and the number of people covered by the insurance. The table below shows a
cost comparison between the different health plans offered by Contoso Electronics:  
<table>
<tr>
<th></th>
<th colspan="2">Employee's cost per paycheck</th>
</tr>
<tr>
<th></th>
<

## Perform a hybrid search + Semantic reranking

In [15]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "What's a performance review?"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    query_type=QueryType.SEMANTIC,  semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=2
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: Performance reviews are<em> conducted annually</em> and are<em> an important part of your career development</em> During the<em> review, your supervisor will discuss your performance over the past year and provide feedback on areas for improvement</em> They will also provide you with an opportunity to discuss your goals and objectives for the upcoming year Performance review...
Semantic Answer Score: 0.90771484375

parent_id: aHR0cHM6Ly9zdGd5b2xhNG5kbDNqZGkuYmxvYi5jb3JlLndpbmRvd3MubmV0L2RvY3VtZW50LWludGVsbGlnZW5jZS1zYW1wbGUtZGF0YS9lbXBsb3llZV9oYW5kYm9vay5wZGY1
chunk_id: 83d4a28b50ce_aHR0cHM6Ly9zdGd5b2xhNG5kbDNqZGkuYmxvYi5jb3JlLndpbmRvd3MubmV0L2RvY3VtZW50LWludGVsbGlnZW5jZS1zYW1wbGUtZGF0YS9lbXBsb3llZV9oYW5kYm9vay5wZGY1_chunks_3
Score: 0.03333333507180214
Content: ### Performance Reviews  
#### Performance Reviews at Contoso Electronics  
At Contoso Electronics, we strive to ensure our employees are getting the feedback they
need to continue growing and developing in thei